In [9]:
import sys
import joblib
import logging
import time
import warnings
import requests
import multiprocessing
import pandas as pd
import numpy as np
import yfinance as yf
import pysentiment as ps

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from zero_shot_learner import extend_df_with_cos_sim
from preprocessor import NewsPreprocessor
from preprocessor import transform_df
from contractions import contractions_dict
from summarizer import Summarizer
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger()
logger.disabled = True

In [10]:
def add_content(url, ratio=0.8):
    """
    Return:
        res_origin: complete paragraph string
        res_ps: important sentence string
        res_bertsum: filtered string by BertSum
    """
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    resp = requests.get(url, timeout=10, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    paragraph = soup.find_all('p')
    paragraph = [p.text for p in paragraph]
    paragraph = paragraph[1:-1]
    res_origin = "".join(paragraph)
    return res_origin


class NewsPreprocessor:
    """
    Data preprocessing class.
    """
    def __init__(self, contractions_dict, lower=True, rm_stopwords=False):
        """
        :param contractions_dict: dict
        :param lower: bool
        :param rm_stopwords: bool
        """
        self.contractions_dict = contractions_dict
        self.lower = lower
        self.rm_stopwords = rm_stopwords

    def remove_unicode(self, text):
        """
        Removes unicode strings like "\u002c" and "x96"
        """
        text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', text)
        text = re.sub(r'[^\x00-\x7f]', r'', text)
        return text

    # Function for expanding contractions
    def expand_contractions(self, text, contractions_dict):
        """
        Finding contractions. (e.g. you've -> you have)
        """
        # Regular expression for finding contractions
        contractions_re = re.compile('(%s)' % '|'.join(self.contractions_dict.keys()))

        def replace(match):
            return contractions_dict[match.group(0)]

        return contractions_re.sub(replace, text)

    def remove_stopwords(self, text):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        text = pattern.sub('', text)
        return text

    def remove_digits(self, text):
        nums = set(map(int, range(10)))
        text = ''.join(i for i in text if i not in nums)
        return text

    def ultimate_clean(self, text):
        if self.lower:
            text = text.lower()
        if self.rm_stopwords:
            text = self.remove_stopwords(text)
        text = self.remove_unicode(text)
        text = self.expand_contractions(text, self.contractions_dict)
        text = self.remove_digits(text)
        return text

In [18]:
# Load data
import swifter
start = time.time()
df = joblib.load("../data/sp500_top100_v1.bin")
df.drop_duplicates(subset="title", inplace=True)
df.drop_duplicates(subset="url", inplace=True)
print("Get content...")
# tqdm.pandas()
# df[["content"]] = df.progress_apply(lambda row: pd.Series(add_content(row["url"])), axis=1)
df_temp2 = pd.DataFrame()
for ticker in df.ticker.unique().tolist():
    print("Start", ticker)
    df_temp = df[df["ticker"] == ticker]
    df_temp["content"] = df_temp["url"].swifter.allow_dask_on_strings().apply(add_content, axis=1)
    df_temp2 = pd.concat([df_temp2, df_temp], axis=0)
    print(df_temp.head(5))
    time.sleep(20)
print("Done!")
print(df)
print(time.time()-start)

joblib.dump(df, "../data/sp500_top100_content_v1.bin", compress=5)

Get content...
Start AAPL



                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 

...


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 


                                               title        date       query  \
0  BRIEF-Apple Inc Says Not Allowing Entertainmen...  2020-03-15  Apple Inc.   
1  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
2  Apple signs multi-year deals with major music ...  2020-03-12  Apple Inc.   
3  Chinese regulators remove 'Plague Inc' game fr...  2020-02-28  Apple Inc.   
4  UPDATE 1-Chinese regulators remove 'Plague Inc...  2020-02-28  Apple Inc.   

                                             url ticker  \
0  https://www.reuters.com/article/idUSFWN2B61K2   AAPL   
1  https://www.reuters.com/article/idUSKBN20Z33J   AAPL   
2  https://www.reuters.com/article/idUSL4N2B54T2   AAPL   
3  https://www.reuters.com/article/idUSKCN20M043   AAPL   
4  https://www.reuters.com/article/idUSL3N2AS0OO   AAPL   

                                             content  
0  March 14 (Reuters) - Apple Inc: * APPLE INC SA...  
1  (Reuters) - Apple Inc has sealed multi-year li... 

['../data/sp500_top100_content_v1.bin']

In [46]:
joblib.dump(df_temp2, "../data/sp500_top100_content_v1.bin", compress=5)
df_t = joblib.load("../data/sp500_top100_content_v1.bin")
df_t

,title,date,query,url,ticker,content
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL,March 14 (Reuters) - Apple Inc: * APPLE INC SA...
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL,(Reuters) - Apple Inc has sealed multi-year li...
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL,(Reuters) - Apple Inc has sealed multi-year li...
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...
...,...,...,...,...,...,...
126654,SAP to streamline hardware infrastructure in d...,2019-11-12,ServiceNow,https://www.reuters.com/article/idUSKBN1XM2IA,NOW,FRANKFURT (Reuters) - German business software...
126655,CORRECTED-SAP to streamline hardware infrastru...,2019-11-12,ServiceNow,https://www.reuters.com/article/idUSL8N27S7C5,NOW,FRANKFURT (Reuters) - German business software...
126659,SAP to return 1.5 bln euros to shareholders in...,2019-11-04,ServiceNow,https://www.reuters.com/article/idUSL3N27K40Z,NOW,FRANKFURT (Reuters) - German business software...
126664,SAP to return 1.5 billion euros to shareholder...,2019-11-04,ServiceNow,https://www.reuters.com/article/idUSKBN1XE2A0,NOW,FRANKFURT (Reuters) - German business software...


In [47]:
def summary(content, ratio=0.5):
    sent_text = nltk.sent_tokenize(content)
    hiv4_function = ps.hiv4.HIV4()
    po = []
    for p in sent_text:
        tokens = hiv4_function.tokenize(p)
        s = hiv4_function.get_score(tokens)
        po.append(s['Polarity'])
    res = []
    for i, p in enumerate(po):
        if(float(p) >= 0.85 or float(p) <= -0.85):
            res.append(sent_text[i])
    res_ps = "".join(res)
#     bert_summarizer = Summarizer(model="distilbert-base-uncased")
#     result = bert_summarizer(content, ratio=ratio)
#     res_bertsum = ''.join(result)
    return res_ps

In [48]:
df_t["ps_content"] = df_t.swifter.allow_dask_on_strings().apply(lambda row: pd.Series(summary(row["content"])), axis=1)

In [50]:
joblib.dump(df_t, "../data/sp500_top100_content_v2.bin", compress=5)

['../data/sp500_top100_content_v2.bin']

In [55]:
from preprocessor import extend_df_with_cos_sim
from preprocessor import transform_df

df = df_t.copy()
preprocessor = NewsPreprocessor(contractions_dict=contractions_dict)
df["clean_ps_content"] = df["ps_content"].swifter.allow_dask_on_strings().apply(lambda x: preprocessor.ultimate_clean(x))
df = extend_df_with_cos_sim(df=df, col="clean_ps_content", labels=["stock", "finance"], sort_by="stock")